In [3]:
import edt
import SimpleITK as sitk
from glob import glob
from pathlib import Path
import json
import pandas as pd
import os
import tqdm
import numpy as np

train_vessel_dir = "/work/vig/Datasets/aneurysm/internal_train/crop_0.4_vessel_v2"
test_vessel_dir = "/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_v2"
external_vessel_dir = "/work/vig/Datasets/aneurysm/external/crop_0.4_vessel_v2"
target_train_vessel_dir = (
    "/work/vig/Datasets/aneurysm/internal_train/crop_0.4_vessel_edt_v2/"
)
target_test_vessel_dir = (
    "/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_edt_v2/"
)

target_external_vessel_dir = (
    "/work/vig/Datasets/aneurysm/external/crop_0.4_vessel_edt_v2"
)

train_im_files = sorted(list(glob(f"{train_vessel_dir}/*")))
val_im_files = sorted(list(glob(f"{test_vessel_dir}/*")))
external_im_files = sorted(list(glob(f"{external_vessel_dir}/*")))

In [4]:
file_paths = train_im_files
save_dir = target_train_vessel_dir

for file in tqdm.tqdm(file_paths[800:]):
    scan_seriesuid = file.split("/")[-1]
    im_header = sitk.ReadImage(file)
    im = sitk.GetArrayFromImage(im_header)
    im = (im == 1).astype(np.uint8)
    im_dist = edt.sdf(im, black_border=False, parallel=8)
    im_dist_header = sitk.GetImageFromArray(im_dist)
    im_dist_header.CopyInformation(im_header)
    save_file_path = os.path.join(save_dir, scan_seriesuid)
    sitk.WriteImage(im_dist_header, save_file_path)

100%|██████████| 386/386 [3:45:04<00:00, 34.99s/it]  


In [ ]:
file_paths = val_im_files
save_dir = target_test_vessel_dir

for file in tqdm.tqdm(file_paths):
    scan_seriesuid = file.split("/")[-1]
    im_header = sitk.ReadImage(file)
    im = sitk.GetArrayFromImage(im_header)
    im = (im == 1).astype(np.uint8)
    im_dist = edt.sdf(im, black_border=False, parallel=52)
    im_dist_header = sitk.GetImageFromArray(im_dist)
    im_dist_header.CopyInformation(im_header)
    save_file_path = os.path.join(save_dir, scan_seriesuid)
    sitk.WriteImage(im_dist_header, save_file_path)

100%|██████████| 152/152 [1:10:26<00:00, 27.81s/it]


In [7]:
im_header = sitk.ReadImage(file)
im_dist = edt.sdf(im, black_border=False, parallel=22)

im_dist_saved = sitk.ReadImage(save_file_path)
im_dist_saved = sitk.GetArrayFromImage(im_dist_saved)

In [8]:
im_dist.shape, im_dist_saved.shape

((425, 507, 507), (425, 507, 507))

In [10]:
(im_dist - im_dist_saved).min()

0.0

In [ ]:
header = sitk.ReadImage(
    "/work/vig/Datasets/aneurysm/internal_train/crop_0.4_vessel_edt/Tr0188.nii.gz"
)